# HeadyRegistry API Demo

> Explore the central catalog: services, workflows, environments, docs, and notebooks.

HeadyRegistry is the brain and directory of the Heady ecosystem.
Every service, workflow, pattern, and environment is registered here.

---

In [ ]:
import requests, json, os

HEADY_ENDPOINT = os.getenv('HEADY_ENDPOINT', 'http://localhost:3300')
HEADY_API_KEY = os.getenv('HEADY_API_KEY', '')
headers = {'x-api-key': HEADY_API_KEY} if HEADY_API_KEY else {}

def api(path):
    return requests.get(f'{HEADY_ENDPOINT}{path}', headers=headers, timeout=15).json()

print(f'Connected to: {HEADY_ENDPOINT}')

## 1. Full Registry Dump

In [ ]:
registry = api('/api/registry')
print(f'Registry Version: {registry.get("registryVersion")}')
print(f'Last Updated: {registry.get("updatedAt")}')
print(f'Total Components: {len(registry.get("components", []))}')
print(f'Total Workflows: {len(registry.get("workflows", []))}')
print(f'Total Environments: {len(registry.get("environments", []))}')
print(f'Total Artifacts: {len(registry.get("artifacts", []))}')
print(f'Total Docs: {len(registry.get("docs", []))}')
print(f'Total Notebooks: {len(registry.get("notebooks", []))}')

## 2. Components by Type

In [ ]:
from collections import Counter

components = registry.get('components', [])
type_counts = Counter(c['type'] for c in components)

print('Component Types:')
for t, count in type_counts.most_common():
    print(f'  {t}: {count}')

print(f'\nActive: {sum(1 for c in components if c.get("status") == "active")}')
print(f'Deprecated: {sum(1 for c in components if c.get("status") == "deprecated")}')

## 3. Lookup a Specific Component

In [ ]:
component_id = 'heady-manager'  # Change this to look up any component
component = api(f'/api/registry/component/{component_id}')
print(json.dumps(component, indent=2))

## 4. Environments

In [ ]:
environments = registry.get('environments', [])
for env in environments:
    status = env.get('status', 'unknown')
    print(f'  [{status}] {env["id"]} - {env.get("endpoint", "n/a")}')

## 5. Workflows

In [ ]:
workflows = registry.get('workflows', [])
for wf in workflows:
    print(f'  {wf["id"]} (owner: {wf.get("owner", "unknown")})')
    print(f'    inputs:  {wf.get("inputs", [])}')
    print(f'    outputs: {wf.get("outputs", [])}')

## 6. Doc Bundles

In [ ]:
docs = registry.get('docs', [])
for d in docs:
    print(f'  [{d.get("status", "?")}] {d["id"]} - {d.get("path", "")}')

---

## Registry as Source of Truth Router

When any agent or UI needs to know what to use, it queries HeadyRegistry:

- "What is the active version of heady-manager in my personal cloud?"
- "Which HCFullPipeline definition applies?"
- "What Arena policy is current for this service?"

*This notebook is maintained under the Checkpoint Protocol.*